<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
#import requests
#from pprint import pprint
#from bs4 import BeautifulSoup
#import lxml.etree as et

import psycopg2
from sqlalchemy import create_engine

In [2]:
# УДАЛИТЬ, но сейчас оставляю для доступа к БД

# параметры подключения 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:

connection = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')


# Юнит 3. Предварительный анализ данных

Здесь и далее:
- query_x_y - переменная в тексте запроса, где x - номер юнита, y - номер задания в юните;
- zx - переманная в результате запроса , где z - буква, алфавитный порядок которой соответсвует номеру задания в юните.

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''select
                count(id)
                from public.vacancies
'''



In [5]:
# результат запроса
a3 = pd.read_sql_query(query_3_1, connection)
a3

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''select
                count(id)
                from public.employers
'''


In [7]:
# результат запроса
b3 = pd.read_sql_query(query_3_2, connection)
b3

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''select
                count(id)
                from public.areas
'''

In [9]:
# результат запроса
c3 = pd.read_sql_query(query_3_3, connection)
c3

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''select
                count(id)
                from public.industries
'''

In [11]:
# результат запроса
d3 = pd.read_sql_query(query_3_4, connection)
d3

,count
0,294


***

При предварительном анализе данных было установлено:
- количество сфер деятельности работодателей - 294, в то же время, для google-запроса "реестр областей и видов профессиональной деятельности" результат будет гораздо обширней, вероятно, данные имеют какой-то специфический "улон";
- количество вакансий превышает количество работодателй, что не противоречит здравому смыслу;
- количество регионов значительно превышает количество субъектов РФ, следовательно, в данных есть другие страны, и/или административно деление более глубокое.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''select
                    count(v.id) cnt,
                    a.name area
                from public.vacancies v
                join public.areas a on v.area_id=a.id   /*объединение таблиц areas и vacancies по id региона*/
                group by a.name
                order by 1 desc
'''


In [13]:
# результат запроса
a4 = pd.read_sql_query(query_4_1, connection)
a4

,cnt,area
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы
...,...,...
764,1,Тарко-Сале
765,1,Новоаннинский
766,1,Бирск
767,1,Сасово


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''select
                count(id)
                from public.vacancies 
                where salary_from is not null or salary_to is not null  /*одной из двух полей с зарплатой не null*/       
'''


In [15]:
# результат запроса
b4 = pd.read_sql_query(query_4_2, connection)
b4

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''select
                    avg(salary_from) avg_from,
                    avg(salary_to) avg_to
                from public.vacancies                 
'''

In [17]:
c4 = pd.read_sql_query(query_4_3, connection)
round(c4)

,avg_from,avg_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''select
                    count(id),
                    schedule,
                    employment
                from public.vacancies 
                group by schedule, employment   /*группировка по типу рабочего графика, затем - по типу трудоустройства*/
                order by count(id) desc                
'''

In [19]:
# результат запроса
d4 = pd.read_sql_query(query_4_4, connection)
d4

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''select
                    experience,
                    count(id)
                from public.vacancies 
                group by experience
                order by 2               
'''

In [21]:
# результат запроса
e4 = pd.read_sql_query(query_4_5, connection)
e4

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Результаты представленных запросов говорят сами за себя. 
В крупных городах вакансий больше, не все работодатели предпочитают указывать зарплатную вилку целиком, тендеция к удаленной работе, которой раньше, вероятно, не наблюдалось и т.д.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''select
                    count(v.id),
                    e.name             
                from public.vacancies v
                join public.employers e on v.employer_id=e.id
                group by e.name
                order by 1 desc
                limit 5 
           
'''

In [23]:
# результат запроса
a5 = pd.read_sql_query(query_5_1, connection)
a5

,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [55]:
# количество работодателй и вакансий по регионам:
query_5_2_1 = f'''select
                    a.name area_name,  /*название региона*/
                    count(v.id) cnt_vac, /*количество вакансий*/
                    count(e.id) cnt_emp  /*количество работодателей*/                          
                from public.areas a
                left join public.employers e on a.id=e.area
                left join public.vacancies v on a.id=v.area_id
                group by a.name
  
'''
# регионы, в которых нет вакансий:
query_5_2_2 = f'''select 
                    a.name area_name,  
                    count(v.id) cnt_vac,
                    count(e.id) cnt_emp                            
                from public.areas a
                left join public.employers e on a.id=e.area
                left join public.vacancies v on a.id=v.area_id
                where v.id is null
                group by a.name
                order by count(e.id) desc
                limit 1
'''



In [56]:
# результат запроса
b5_1 = pd.read_sql_query(query_5_2_1, connection)
display(b5_1)

b5_2 = pd.read_sql_query(query_5_2_2, connection)
display(b5_2)

,area_name,cnt_vac,cnt_emp
0,Анна,1,0
1,Орлов,0,0
2,Пыть-Ях,5,5
3,Бодайбо,10,10
4,Шацк (Рязанская область),0,0
...,...,...,...
1357,Новый Оскол,0,0
1358,Павлово,4,4
1359,Ильинка (Икрянинский район),0,0
1360,Чамзинка,0,0


,area_name,cnt_vac,cnt_emp
0,Россия,0,410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса
query_5_3 = f'''select
                    e.name employer_name,  
                    count(distinct v.area_id) cnt_area                                                        
                from public.vacancies v
                join public.employers e on v.employer_id=e.id
                group by e.id
                order by 2 desc
  
'''


In [27]:
# результат запроса
c5 = pd.read_sql_query(query_5_3, connection)
c5

,employer_name,cnt_area
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [57]:
# текст запроса #count(employer_id) cnt_emp   
query_5_4 = f'''select
                count(e.id)                                                                                        
                from public.employers e
                left join public.employers_industries ei on e.id=ei.employer_id
                where ei.industry_id is null          
  
'''

In [58]:
# результат запроса
d5 = pd.read_sql_query(query_5_4, connection)
d5

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса #count(employer_id) cnt_emp   
query_5_5 = f'''select
                    e.name,
                    count(ei.industry_id)                                                                                        
                from public.employers e
                join public.employers_industries ei on e.id=ei.employer_id
                group by e.id
                having count(ei.industry_id)=4
                order by 1 
             
  
'''

In [31]:
# результат запроса
e5 = pd.read_sql_query(query_5_5, connection)
e5

,name,count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1133,ЮРИОН,4
1134,ЮТИП Технологии,4
1135,ЯКласс,4
1136,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [59]:
# текст запроса 
query_5_6 = f'''select
                count(e.id)                                                                                                    
                from public.employers_industries ei
                join public.employers e on e.id=ei.employer_id
                join public.industries i on i.id=ei.industry_id
                where i.name = 'Разработка программного обеспечения'                            

'''

In [60]:
# результат запроса
f5 = pd.read_sql_query(query_5_6, connection)
f5

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# код для получения списка городов-милионников

url = ("https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8")
all_tables = pd.read_html(url)
tab1=all_tables[1]
cities=list(tab1['Город'])
cities[-3]='Воронеж' # замена 'Воронеж[а]' на 'Воронеж'

cities=tuple(cities) # кортеж лучше воспринимается sql
print(cities)





('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [35]:
# текст запроса
query_5_7 = f'''select
                    a.name, 
                    count(v.id) cnt_vacancies                                                                                                                  
                from public.vacancies v
                join public.areas a on v.area_id=a.id
                join public.employers e on v.employer_id=e.id
                where a.name IN {cities} and e.name like 'Яндекс'
                group by  a.name
                

                UNION ALL
                
                select
                    'total', 
                    count('cnt_vacancies')                                                                                                                  
                from public.vacancies v
                join public.areas a on v.area_id=a.id
                join public.employers e on v.employer_id=e.id
                where a.name IN {cities} and e.name like 'Яндекс'
                
                order by 2            
                                
'''


In [36]:
# результат запроса
g5 = pd.read_sql_query(query_5_7, connection)
g5

,name,cnt_vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

В представленной базе данных Яндекс - лидер. Большего всего вакансий, большего всего регионов, в которых вакансии представлены. Много людей пользуются яндекс-картами, яндекс-такси, каршерингом и прочими "продуктами экосистемы". Странно, что нет СБЕРа, у которого тоже много различных сервисов. Вероятно, СБЕР испытывает непростой период из-за 100-го пакета санкций.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [38]:
# текст запроса
query_6_1 = f'''select
                name      
                from public.vacancies
                where lower(name) like '%%data%%' or lower(name) like '%%данн%%' /*lower для нечувствительности к регистру*/
         
'''

In [39]:
# результат запроса
a6 = pd.read_sql_query(query_6_1, connection)
a6

,name
0,Аналитик хранилищ данных и BI-систем/Аналитик ...
1,Data-analysts (удаленно)
2,Стажер Data Engineer/Data Аналитик в команду Б...
3,Старший специалист хранилищ данных
4,Администратор баз данных и сетей
...,...
1766,Программист Delphi / разработчик баз данных
1767,Разработчик баз данных (Big Data)
1768,Аналитик данных
1769,Data аналитик /Аналитик BI (удаленно)


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [40]:
# текст запроса
query_6_2 = f'''select
                count(id)      
                from public.vacancies
                where (lower(name) like '%%data scientist%%' or 
                    lower(name) like '%%data science%%' or
                    lower(name) like '%%исследователь данных%%' or
                    (name like '%%ML%%' and name not like '%%HTML%%') or
                    lower(name) like '%%machine learning%%' or
                    lower(name) like '%%машинн%%обучен%%') 
                    and
                    (lower(name) like '%%junior%%' or
                    lower(experience) like '%%нет опыта%%' or
                    lower(employment) like '%%стажировка%%')
        
'''

In [41]:
# результат запроса
b6 = pd.read_sql_query(query_6_2, connection)
b6

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [42]:
# текст запроса
query_6_3 = f'''select
                count(id)      
                from public.vacancies
                where (lower(name) like '%%data scientist%%' or 
                    lower(name) like '%%data science%%' or
                    lower(name) like '%%исследователь данных%%' or
                    (name like '%%ML%%' and name not like '%%HTML%%') or
                    lower(name) like '%%machine learning%%' or
                    lower(name) like '%%машинн%%обучен%%') 
                    and
                    (lower(key_skills) like '%%sql%%' or 
                    lower(key_skills) like '%%postgres%%')
       
'''

In [43]:
# результат запроса
c6 = pd.read_sql_query(query_6_3, connection)
c6

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [44]:
# текст запроса
query_6_4 = f'''select
                count(*)
                from public.vacancies
                where (lower(name) like '%%data scientist%%' or 
                    lower(name) like '%%data science%%' or
                    lower(name) like '%%исследователь данных%%' or
                    (name like '%%ML%%' and name not like '%%HTML%%') or
                    lower(name) like '%%machine learning%%' or
                    lower(name) like '%%машинн%%обучен%%') 
                    and
                    (lower(key_skills) like '%%python%%')
       
'''

In [45]:
# результат запроса
d6 = pd.read_sql_query(query_6_4, connection)
d6

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [46]:
# текст запроса
query_6_5 = f'''select
                avg(length(key_skills)-length(replace(key_skills,'\t',''))+1) 
                from public.vacancies
                where (lower(name) like '%%data scientist%%' or 
                    lower(name) like '%%data science%%' or
                    lower(name) like '%%исследователь данных%%' or
                    (name like '%%ML%%' and name not like '%%HTML%%') or
                    lower(name) like '%%machine learning%%' or
                    lower(name) like '%%машинн%%обучен%%') 

       
'''

In [47]:
# результат запроса
e6 = pd.read_sql_query(query_6_5, connection)
round(e6,2)

,avg
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [48]:
# текст запроса
query_6_6 = f'''select
                    (salary_from + salary_to)/2 avg_sal
                from public.vacancies
                where (lower(name) like '%%data scientist%%' or 
                    lower(name) like '%%data science%%' or
                    lower(name) like '%%исследователь данных%%' or
                    (name like '%%ML%%' and name not like '%%HTML%%') or
                    lower(name) like '%%machine learning%%' or
                    lower(name) like '%%машинн%%обучен%%')
                    and
                    (salary_to is not null and salary_from is not null)
                    and (experience like 'От 3 до 6 лет')
                
                UNION ALL
                
                select
                    salary_from avg_sal
                from public.vacancies 
                where (lower(name) like '%%data scientist%%' or 
                    lower(name) like '%%data science%%' or
                    lower(name) like '%%исследователь данных%%' or
                    (name like '%%ML%%' and name not like '%%HTML%%') or
                    lower(name) like '%%machine learning%%' or
                    lower(name) like '%%машинн%%обучен%%')
                    and
                    (salary_to is null and salary_from is not null)
                    and (experience like 'От 3 до 6 лет')
                                       
                    UNION ALL
                
                select
                    salary_to avg_sal
                from public.vacancies
                where (lower(name) like '%%data scientist%%' or 
                    lower(name) like '%%data science%%' or
                    lower(name) like '%%исследователь данных%%' or
                    (name like '%%ML%%' and name not like '%%HTML%%') or
                    lower(name) like '%%machine learning%%' or
                    lower(name) like '%%машинн%%обучен%%')
                    and
                    (salary_to is not null and salary_from is null)
                    and (experience like 'От 3 до 6 лет')
                    
                                   
       
'''

In [49]:
# результат запроса
f6 = pd.read_sql_query(query_6_6, connection)
display(f6)
round(f6.mean())


,avg_sal
0,450000
1,250000
2,233794
3,119000
4,150000
5,200000
6,300000
7,140000
8,225000
9,275000


avg_sal    243115.0
dtype: float64

***

Навыки SQL и PYTHON однозначно важны для вакансий, имеющих отношение к данным. Принимая во внимание, что начинающему DS непросто найти работу, освоение этих навыком может иметь рещающее значение. С учетом неполохой средней ЗП освоение любых требуемых навыков целесообразно и выгодно.

# Общий вывод по проекту

В целом предыдущие выводы отображают текущую картину рынка вакансий. Конечно, было бы интересно "поиграть" с навыками и зарплатами, посмотреть, какие навыки больше ценятся. 
В текущей изменчивой обстановке результаты проведенного анализа могут характеризовать работодателей с наибольшим количеством вакансий, как усточивых к кризису. Однако наличие сложной ситуации на рынке и в мире не оказывает влияния на классические проблемы и тендеции, такие как, например, урбанизация. 

Данные могут быть использованы для дальейшей работы, условно, кадрового агентства.

In [52]:
# Закрыть соединение
connection.connect().close()
